In [ ]:
# | default_exp _docusaurus_helper

# Docusaurus Helper

In [ ]:
# | export

import itertools
import re
import types
from inspect import getmembers, isclass, isfunction, signature
from pathlib import Path
from typing import *

import typer
from docstring_parser import parse
from docstring_parser.common import DocstringParam, DocstringRaises, DocstringReturns
from nbdev.config import get_config
from nbdev_mkdocs.mkdocs import (
    _add_all_submodules,
    _get_api_summary,
    _import_all_members,
    _import_functions_and_classes,
    _import_submodules,
)

In [ ]:
from tempfile import TemporaryDirectory

from typer.testing import CliRunner
from pydantic import BaseModel

In [ ]:
# | export


def _format_docstring_sections(
    items: Union[List[DocstringParam], List[DocstringReturns], List[DocstringRaises]],
    keyword: str,
) -> str:
    """Format a list of docstring sections

    Args:
        items: A list of DocstringParam objects
        keyword: The type of section to format (e.g. 'Parameters', 'Returns', 'Exceptions')

    Returns:
        The formatted docstring.
    """
    formatted_docstring = ""
    if len(items) > 0:
        formatted_docstring += f"**{keyword}**:\n"
        for item in items:
            if keyword == "Parameters":
                formatted_docstring += f"- `{item.arg_name}`: {item.description}\n"  # type: ignore
            elif keyword == "Exceptions":
                formatted_docstring += f"- `{item.type_name}`: {item.description}\n"
            else:
                formatted_docstring += f"- {item.description}\n"
        formatted_docstring = f"{formatted_docstring}\n"
    return formatted_docstring

In [ ]:
fixture = parse(
    """
This is a docstring for a sample function.

It can contain multiple lines and can include *markdown* syntax.

Args:
    name: name of the person
    age: age of the person
    
Returns:
    A formatted string

Raises:
    ValueError: If name is not a string
    TypeError: If name is not a string
"""
)

actual = _format_docstring_sections(fixture.params, "Parameters")
expected = """**Parameters**:
- `name`: name of the person
- `age`: age of the person

"""

print(actual)
assert actual == expected

**Parameters**:
- `name`: name of the person
- `age`: age of the person




In [ ]:
actual = _format_docstring_sections(fixture.many_returns, "Returns")
expected = """**Returns**:
- A formatted string

"""

print(actual)
assert actual == expected

**Returns**:
- A formatted string




In [ ]:
actual = _format_docstring_sections(fixture.raises, "Exceptions")
expected = """**Exceptions**:
- `ValueError`: If name is not a string
- `TypeError`: If name is not a string

"""

print(actual)
assert actual == expected

**Exceptions**:
- `ValueError`: If name is not a string
- `TypeError`: If name is not a string




In [ ]:
# | export


def _docstring_to_markdown(docstring: str) -> str:
    """Converts a docstring to a markdown-formatted string.

    Args:
        docstring: The docstring to convert.

    Returns:
        The markdown-formatted docstring.
    """
    parsed_docstring = parse(docstring)
    formatted_docstring = f"{parsed_docstring.short_description}\n\n"
    formatted_docstring += (
        f"{parsed_docstring.long_description}\n\n"
        if parsed_docstring.long_description
        else ""
    )
    formatted_docstring += _format_docstring_sections(
        parsed_docstring.params, "Parameters"
    )
    formatted_docstring += _format_docstring_sections(
        parsed_docstring.many_returns, "Returns"
    )
    formatted_docstring += _format_docstring_sections(
        parsed_docstring.raises, "Exceptions"
    )

    return formatted_docstring

In [ ]:
fixture = """This is a docstring for a sample function."""

expected = """This is a docstring for a sample function.

"""

actual = _docstring_to_markdown(fixture)
print(actual)

assert actual == expected

This is a docstring for a sample function.




In [ ]:
fixture = """This is a docstring for a sample function.

Args:
    name: name of the person
    age: age of the person
"""

expected = """This is a docstring for a sample function.

**Parameters**:
- `name`: name of the person
- `age`: age of the person

"""

actual = _docstring_to_markdown(fixture)
print(actual)

assert actual == expected

This is a docstring for a sample function.

**Parameters**:
- `name`: name of the person
- `age`: age of the person




In [ ]:
fixture = """
This is a docstring for a sample function.

It can contain multiple lines and can include *markdown* syntax.

Args:
    name: name of the person
    age: age of the person
    
Returns:
    A formatted string

Raises:
    ValueError: If name is not a string
    TypeError: If name is not a string
"""

expected = """This is a docstring for a sample function.

It can contain multiple lines and can include *markdown* syntax.

**Parameters**:
- `name`: name of the person
- `age`: age of the person

**Returns**:
- A formatted string

**Exceptions**:
- `ValueError`: If name is not a string
- `TypeError`: If name is not a string

"""

actual = _docstring_to_markdown(fixture)
print(actual)

assert actual == expected

This is a docstring for a sample function.

It can contain multiple lines and can include *markdown* syntax.

**Parameters**:
- `name`: name of the person
- `age`: age of the person

**Returns**:
- A formatted string

**Exceptions**:
- `ValueError`: If name is not a string
- `TypeError`: If name is not a string




In [ ]:
# | export


def _get_submodules(module_name: str) -> List[str]:
    """Get a list of all submodules contained within the module.

    Args:
        module_name: The name of the module to retrieve submodules from

    Returns:
        A list of submodule names within the module
    """
    members = _import_all_members(module_name)
    members_with_submodules = _add_all_submodules(members)
    members_with_submodules_str: List[str] = [
        x[:-1] if x.endswith(".") else x for x in members_with_submodules
    ]
    return members_with_submodules_str

In [ ]:
module_name = "fastkafka"
members_with_submodules = _get_submodules(module_name)
members_with_submodules

['fastkafka',
 'fastkafka.FastKafka',
 'fastkafka.KafkaEvent',
 'fastkafka.testing',
 'fastkafka.testing.LocalKafkaBroker',
 'fastkafka.testing.LocalRedpandaBroker',
 'fastkafka.testing.Tester']

In [ ]:
# | export


def _load_submodules(
    module_name: str, members_with_submodules: List[str]
) -> List[Union[types.FunctionType, Type[Any]]]:
    """Load the given submodules from the module.

    Args:
        module_name: The name of the module whose submodules to load
        members_with_submodules: A list of submodule names to load

    Returns:
        A list of imported submodule objects.
    """
    submodules = _import_submodules(module_name)
    members: List[Tuple[str, Union[types.FunctionType, Type[Any]]]] = list(
        itertools.chain(*[_import_functions_and_classes(m) for m in submodules])
    )
    names = [
        y
        for x, y in members
        if f"{y.__module__}.{y.__name__}" in members_with_submodules
    ]
    return names

In [ ]:
module_name = "fastkafka"
members_with_submodules = _get_submodules(module_name)
symbols = _load_submodules(module_name, members_with_submodules)
symbols

[fastkafka.FastKafka,
 fastkafka.KafkaEvent,
 fastkafka.testing.LocalKafkaBroker,
 fastkafka.testing.LocalRedpandaBroker,
 fastkafka.testing.Tester]

In [ ]:
# | export

def _convert_union_to_optional(annotation_str: str) -> str:
    """Convert the 'Union[Type1, Type2, ..., NoneType]' to 'Optional[Type1, Type2, ...]' in the given annotation string

    Args:
        annotation_str: The type annotation string to convert.

    Returns:
        The converted type annotation string.
    """
    pattern = r'Union\[(.*)?,\s*NoneType\s*\]'
    match = re.search(pattern, annotation_str)
    if match:
        union_type = match.group(1)
        optional_type = f"Optional[{union_type}]"
        return re.sub(pattern, optional_type, annotation_str)
    else:
        return annotation_str

In [ ]:
fixtures = [
    {
        "input": "arg_1: Union[int, NoneType] = 80",
        "expected": "arg_1: Optional[int] = 80",
    },
    {
        "input": "arg_1: Union[Dict[str, str], NoneType]",
        "expected": "arg_1: Optional[Dict[str, str]]",
    },
    {
        "input": "arg_1: Union[Dict[str, str], str]",
        "expected": "arg_1: Union[Dict[str, str], str]",
    },
    {
        "input": "arg_1: str",
        "expected": "arg_1: str",
    },
    {
        "input": "arg_1: bool = False",
        "expected": "arg_1: bool = False",
    },
    {
        "input": "prefix: str = 'to_'",
        "expected": "prefix: str = 'to_'",
    },
    
]

for fixture in fixtures:
    actual = _convert_union_to_optional(fixture["input"])
    print(actual)
    assert actual == fixture["expected"]

arg_1: Optional[int] = 80
arg_1: Optional[Dict[str, str]]
arg_1: Union[Dict[str, str], str]
arg_1: str
arg_1: bool = False
prefix: str = 'to_'


In [ ]:
# | export

def _get_arg_list_with_signature(_signature: signature) -> str:
    """Converts a function's signature into a string representation of its argument list.

    Args:
        _signature (signature): The signature object for the function to convert.

    Returns:
        str: A string representation of the function's argument list.
    """
    arg_list = []
    for param in _signature.parameters.values():
        arg_list.append(_convert_union_to_optional(str(param)))

    return ", ".join(arg_list)

In [ ]:
def fixture_function(
    arg_1: str, arg_2, arg_3: Union[Dict[str, str], str], arg_4: Optional[int] = 80
) -> str:
    pass

_signature = signature(fixture_function)

expected = "arg_1: str, arg_2, arg_3: Union[Dict[str, str], str], arg_4: Optional[int] = 80"
actual = _get_arg_list_with_signature(_signature)

print(actual)
assert actual == expected

arg_1: str, arg_2, arg_3: Union[Dict[str, str], str], arg_4: Optional[int] = 80


In [ ]:
def fixture_function(arg_1: str, arg_2) -> None:
    pass

_signature = signature(fixture_function)

expected = "arg_1: str, arg_2"
actual = _get_arg_list_with_signature(_signature)

print(actual)
assert actual == expected

arg_1: str, arg_2


In [ ]:
# | export


def _get_symbol_definition(symbol: Union[types.FunctionType, Type[Any]]) -> str:
    """Return the definition of a given symbol.

    Args:
        symbol: A function or method object to get the definition for.

    Returns:
        A string representing the function definition
    """
    _signature = signature(symbol)
    arg_list = _get_arg_list_with_signature(_signature)
    ret_val = ""

    if isfunction(symbol):
        ret_val = f"`def {symbol.__name__}({arg_list})"
        if _signature.return_annotation and "inspect._empty" not in str(
            _signature.return_annotation
        ):
            if isinstance(_signature.return_annotation, type):
                ret_val = ret_val + f" -> {_signature.return_annotation.__name__}`\n"
            else:
                ret_val = ret_val + f" -> {_signature.return_annotation}`\n"

        else:
            ret_val = ret_val + " -> None`\n"

    return ret_val

In [ ]:
TestCallable = Callable[[BaseModel], Union[Awaitable[None], None]]


def fixture_function(arg_1: str) -> TestCallable:
    pass


actual = _get_symbol_definition(fixture_function)
print(actual)
assert 'def fixture_function(arg_1: str) -> typing.Callable[[pydantic.main.BaseModel]' in actual

`def fixture_function(arg_1: str) -> typing.Callable[[pydantic.main.BaseModel], typing.Union[NoneType, typing.Awaitable[NoneType]]]`



In [ ]:
def fixture_function(arg_1: str, arg_2) -> None:
    pass


actual = _get_symbol_definition(fixture_function)
expected = "`def fixture_function(arg_1: str, arg_2) -> None`\n"

print(actual)
assert actual == expected

`def fixture_function(arg_1: str, arg_2) -> None`



In [ ]:
def fixture_function(arg_1: str, arg_2) -> int:
    pass


actual = _get_symbol_definition(fixture_function)
expected = "`def fixture_function(arg_1: str, arg_2) -> int`\n"

print(actual)
assert actual == expected

`def fixture_function(arg_1: str, arg_2) -> int`



In [ ]:
def fixture_function(arg_1: str, arg_2) -> "Tester":
    pass


actual = _get_symbol_definition(fixture_function)
expected = '`def fixture_function(arg_1: str, arg_2) -> Tester`\n'

print(actual)
assert actual == expected

`def fixture_function(arg_1: str, arg_2) -> Tester`



In [ ]:
def fixture_function(
    arg_1: int,
    arg_2: str = "default_string",
    arg_3: Dict[str, int] = {},
    arg_4: Optional[float] = None,
    arg_5: Tuple[int, str, float] = (1, "string", 2.0),
    arg_6: List[Union[int, str]] = [1, "string"],
    arg_7: Set[int] = {1, 2, 3},
    arg_8: Union[int, str] = "string"
) -> None:
    pass


actual = _get_symbol_definition(fixture_function)
expected = "`def fixture_function(arg_1: int, arg_2: str = 'default_string', arg_3: Dict[str, int] = {}, arg_4: Optional[float] = None, arg_5: Tuple[int, str, float] = (1, 'string', 2.0), arg_6: List[Union[int, str]] = [1, 'string'], arg_7: Set[int] = {1, 2, 3}, arg_8: Union[int, str] = 'string') -> None`\n"

print(actual)
assert actual == expected

`def fixture_function(arg_1: int, arg_2: str = 'default_string', arg_3: Dict[str, int] = {}, arg_4: Optional[float] = None, arg_5: Tuple[int, str, float] = (1, 'string', 2.0), arg_6: List[Union[int, str]] = [1, 'string'], arg_7: Set[int] = {1, 2, 3}, arg_8: Union[int, str] = 'string') -> None`



In [ ]:
# | export


def _get_formatted_docstring_for_symbol(
    symbol: Union[types.FunctionType, Type[Any]]
) -> str:
    """Recursively parses and get formatted docstring of a symbol.

    Args:
        symbol: A Python class or function object to parse the docstring for.

    Returns:
        A formatted docstring of the symbol and its members.

    """

    def traverse(symbol: Union[types.FunctionType, Type[Any]], contents: str) -> str:
        """Recursively traverse the members of a symbol and append their docstrings to the provided contents string.

        Args:
            symbol: A Python class or function object to parse the docstring for.
            contents: The current formatted docstrings.

        Returns:
            The updated formatted docstrings.

        """
        for x, y in getmembers(symbol):
            if not x.startswith("_") or x.endswith("__"):
                if isfunction(y) and y.__doc__ is not None:
                    contents += f"{_get_symbol_definition(y)}\n{_docstring_to_markdown(y.__doc__)}"
                elif isclass(y) and not x.startswith("__") and y.__doc__ is not None:
                    contents += f"{_get_symbol_definition(y)}\n{_docstring_to_markdown(y.__doc__)}"
                    contents = traverse(y, contents)
        return contents

    contents = (
        f"{_get_symbol_definition(symbol)}\n{_docstring_to_markdown(symbol.__doc__)}"
        if symbol.__doc__ is not None
        else ""
    )
    if isclass(symbol):
        contents = traverse(symbol, contents)
    return contents

In [ ]:
def fixture_function(arg_1: str, arg_2: Union[List[str], str], arg_3: Optional[int], arg_4: Optional[str] = None) -> str:
    """This is a one line description for the function

    Args:
        arg_1: Argument 1
        arg_2: Argument 2
        arg_3: Argument 3
        arg_4: Argument 4

    Returns:
        The concatinated string
    """
    pass


expected = """`def fixture_function(arg_1: str, arg_2: Union[List[str], str], arg_3: Optional[int], arg_4: Optional[str] = None) -> str`

This is a one line description for the function

**Parameters**:
- `arg_1`: Argument 1
- `arg_2`: Argument 2
- `arg_3`: Argument 3
- `arg_4`: Argument 4

**Returns**:
- The concatinated string

"""

actual = _get_formatted_docstring_for_symbol(fixture_function)
print(actual)

assert actual == expected

`def fixture_function(arg_1: str, arg_2: Union[List[str], str], arg_3: Optional[int], arg_4: Optional[str] = None) -> str`

This is a one line description for the function

**Parameters**:
- `arg_1`: Argument 1
- `arg_2`: Argument 2
- `arg_3`: Argument 3
- `arg_4`: Argument 4

**Returns**:
- The concatinated string




In [ ]:
class Vehicle:
    """This is a docstring for the class"""

    def __init__(self, brand: str, model: str, type: str):
        """Constructor

        Args:
            brand: Name of the brand
            model: Name of the model
            type: Model type
        """
        self.brand = brand
        self.model = model
        self.type = type
        self.gas_tank_size = 14
        self.fuel_level = 0

    def fuel_up(self):
        """Fuel up"""
        self.fuel_level = self.gas_tank_size
        print("Gas tank is now full.")

    def drive(self):
        """Drive"""
        print(f"The {self.model} is now driving.")


expected = """
This is a docstring for the class

`def __init__(self, brand: str, model: str, type: str) -> None`

Constructor

**Parameters**:
- `brand`: Name of the brand
- `model`: Name of the model
- `type`: Model type

`def drive(self) -> None`

Drive

`def fuel_up(self) -> None`

Fuel up

"""

actual = _get_formatted_docstring_for_symbol(Vehicle)
print(actual)

assert actual == expected


This is a docstring for the class

`def __init__(self, brand: str, model: str, type: str) -> None`

Constructor

**Parameters**:
- `brand`: Name of the brand
- `model`: Name of the model
- `type`: Model type

`def drive(self) -> None`

Drive

`def fuel_up(self) -> None`

Fuel up




In [ ]:
class Outer:
    """Outer Class"""

    def __init__(self):
        """Outer class constructor"""
        ## instantiating the 'Inner' class
        self.inner = self.Inner()

    def reveal(self):
        """Reveal function"""
        ## calling the 'Inner' class function display
        self.inner.inner_display("Calling Inner class function from Outer class")

    class Inner:
        """Inner Class"""

        def inner_display(self, msg):
            """Inner display"""
            print(msg)

    class Inner2:
        """Inner2 Class"""

        def inner_display_2(self, msg):
            """Inner display_2"""
            print(msg)


expected = """
Outer Class


Inner Class

`def inner_display(self, msg) -> None`

Inner display


Inner2 Class

`def inner_display_2(self, msg) -> None`

Inner display_2

`def __init__(self) -> None`

Outer class constructor

`def reveal(self) -> None`

Reveal function

"""

actual = _get_formatted_docstring_for_symbol(Outer)
print(actual)

assert actual == expected


Outer Class


Inner Class

`def inner_display(self, msg) -> None`

Inner display


Inner2 Class

`def inner_display_2(self, msg) -> None`

Inner display_2

`def __init__(self) -> None`

Outer class constructor

`def reveal(self) -> None`

Reveal function




In [ ]:
# | export


def _convert_html_style_attribute_to_jsx(contents: str) -> str:
    """Converts the inline style attributes in an HTML string to JSX compatible format.

    Args:
        contents: A string containing an HTML document or fragment.

    Returns:
        A string with inline style attributes converted to JSX compatible format.
    """
    style_regex = re.compile(r'style="(.+?)"')
    style_matches = style_regex.findall(contents)

    for style_match in style_matches:
        style_dict = {}
        styles = style_match.split(";")
        for style in styles:
            key_value = style.split(":")
            if len(key_value) == 2:
                key = re.sub(
                    r"-(.)", lambda m: m.group(1).upper(), key_value[0].strip()
                )
                value = key_value[1].strip().replace("'", '"')
                style_dict[key] = value
        replacement = "style={{"
        for key, value in style_dict.items():
            replacement += f"{key}: '{value}', "
        replacement = replacement[:-2] + "}}"
        contents = contents.replace(f'style="{style_match}"', replacement)

    return contents

In [ ]:
fixtures = [
    {
        "input": """<a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style="float:right; font-size:smaller">source</a> some text goes here <a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style="float:right; font-size:smaller">source</a>""",
        "expected": """<a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style={{float: 'right', fontSize: 'smaller'}}>source</a> some text goes here <a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style={{float: 'right', fontSize: 'smaller'}}>source</a>""",
    },
    {
        "input": '<span style="color: red;">Test</span>',
        "expected": "<span style={{color: 'red'}}>Test</span>",
    },
    {
        "input": '<div style="background-color: blue; border: 1px solid black;">Test</div>',
        "expected": "<div style={{backgroundColor: 'blue', border: '1px solid black'}}>Test</div>",
    },
    {
        "input": '<span style="font-size: 1.2rem;">Test</span>',
        "expected": "<span style={{fontSize: '1.2rem'}}>Test</span>",
    },
    {
        "input": """<pre style="white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace"></pre>""",
        "expected": """<pre style={{whiteSpace: 'pre', overflowX: 'auto', lineHeight: 'normal', fontFamily: 'Menlo,"DejaVu Sans Mono",consolas,"Courier New",monospace'}}></pre>""",
    },
]

for fixture in fixtures:
    actual = _convert_html_style_attribute_to_jsx(fixture["input"])
    print("*" * 120)
    print(actual)
    assert actual == fixture["expected"], fixture["expected"]

************************************************************************************************************************
<a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style={{float: 'right', fontSize: 'smaller'}}>source</a> some text goes here <a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style={{float: 'right', fontSize: 'smaller'}}>source</a>
************************************************************************************************************************
<span style={{color: 'red'}}>Test</span>
************************************************************************************************************************
<div style={{backgroundColor: 'blue', border: '1px solid black'}}>Test</div>
************************************************************************************************************************
<span style={{fontSize: '1

In [ ]:
# | export


def _get_all_markdown_files_path(docs_path: Path) -> List[Path]:
    """Get all Markdown files in a directory and its subdirectories.

    Args:
        directory: The path to the directory to search in.

    Returns:
        A list of paths to all Markdown files found in the directory and its subdirectories.
    """
    markdown_files = [file_path for file_path in docs_path.glob("**/*.md")]
    return markdown_files

In [ ]:
with TemporaryDirectory() as d:
    module_name = "fastkafka"

    docs_path = Path(d) / "docusaurus" / "docs"
    docs_path.mkdir(parents=True)

    api_path = docs_path / "api"
    api_path.mkdir(parents=True)

    blog_path = docs_path / "blog"
    blog_path.mkdir(parents=True)

    nested_api_path = api_path / "fastKafka"
    nested_api_path.mkdir(parents=True)

    for p in [docs_path, api_path, blog_path, nested_api_path]:
        with open((p / "file.md"), "w") as f:
            f.write("sample text")

    actual = _get_all_markdown_files_path(docs_path)
    expected = [
        Path(docs_path) / "file.md",
        Path(api_path) / "file.md",
        Path(nested_api_path) / "file.md",
        Path(blog_path) / "file.md",
    ]

    print(actual)
    assert sorted(actual) == sorted(expected), expected

[Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpqewgtogo/docusaurus/docs/file.md'), Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpqewgtogo/docusaurus/docs/blog/file.md'), Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpqewgtogo/docusaurus/docs/api/file.md'), Path('/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpqewgtogo/docusaurus/docs/api/fastKafka/file.md')]


In [ ]:
# | export


def _fix_special_symbols_in_html(contents: str) -> str:
    contents = contents.replace("”", '"')
    return contents

In [ ]:
fixture = """<a href="https://colab.research.google.com/github/airtai/fastkafka/blob/main/nbs/guides/Guide_00_FastKafka_Demo.ipynb" target=”_blank”>"""
expected = """<a href="https://colab.research.google.com/github/airtai/fastkafka/blob/main/nbs/guides/Guide_00_FastKafka_Demo.ipynb" target="_blank">"""

actual = _fix_special_symbols_in_html(fixture)
print(actual)
assert actual == expected

<a href="https://colab.research.google.com/github/airtai/fastkafka/blob/main/nbs/guides/Guide_00_FastKafka_Demo.ipynb" target="_blank">


In [ ]:
# | export


def _fix_invalid_syntax_in_markdown(docs_path: Path) -> None:
    """Fix invalid HTML syntax in markdown files and converts inline style attributes to JSX-compatible format.

    Args:
        docs_path: The path to the root directory to search for markdown files.
    """
    markdown_files = _get_all_markdown_files_path(docs_path)
    updated_contents = [
        _convert_html_style_attribute_to_jsx(Path(file).read_text())
        for file in markdown_files
    ]
    updated_contents = [
        _fix_special_symbols_in_html(contents) for contents in updated_contents
    ]
    for i, file_path in enumerate(markdown_files):
        file_path.write_text(updated_contents[i])

In [ ]:
with TemporaryDirectory() as d:
    module_name = "fastkafka"

    docs_path = Path(d) / "docusaurus" / "docs"
    docs_path.mkdir(parents=True)

    api_path = docs_path / "api"
    api_path.mkdir(parents=True)

    blog_path = docs_path / "blog"
    blog_path.mkdir(parents=True)

    nested_api_path = api_path / "fastKafka"
    nested_api_path.mkdir(parents=True)

    for p in [docs_path, api_path, blog_path, nested_api_path]:
        with open((p / "file.md"), "w") as f:
            f.write(
                """<a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target=”_blank” style="float:right; font-size:smaller">source</a> some text goes here <span style="color: red;">Test</span> and one more tag <div style="background-color: blue; border: 1px solid black;">Test</div>"""
            )

    _fix_invalid_syntax_in_markdown(docs_path)
    expected = """<a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style={{float: 'right', fontSize: 'smaller'}}>source</a> some text goes here <span style={{color: 'red'}}>Test</span> and one more tag <div style={{backgroundColor: 'blue', border: '1px solid black'}}>Test</div>"""

    for p in [docs_path, api_path, blog_path, nested_api_path]:
        with open((p / "file.md"), "r") as f:
            actual = f.read()
            print("*" * 120)
            print(actual)
            assert actual == expected

************************************************************************************************************************
<a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style={{float: 'right', fontSize: 'smaller'}}>source</a> some text goes here <span style={{color: 'red'}}>Test</span> and one more tag <div style={{backgroundColor: 'blue', border: '1px solid black'}}>Test</div>
************************************************************************************************************************
<a
href="https://github.com/airtai/fastkafka/blob/main/fastkafka/_components/test_dependencies.py#L28"
target="_blank" style={{float: 'right', fontSize: 'smaller'}}>source</a> some text goes here <span style={{color: 'red'}}>Test</span> and one more tag <div style={{backgroundColor: 'blue', border: '1px solid black'}}>Test</div>
************************************************************************************************

In [ ]:
# | export


def _generate_markdown_docs(module_name: str, docs_path: Path) -> None:
    """Generates Markdown documentation files for the symbols in the given module and save them to the given directory.

    Args:
        module_name: The name of the module to generate documentation for.
        docs_path: The path to the directory where the documentation files will be saved.
    """
    members_with_submodules = _get_submodules(module_name)
    symbols = _load_submodules(module_name, members_with_submodules)

    for symbol in symbols:
        content = f"`{symbol.__module__}.{symbol.__name__}`\n\n"
        content += _get_formatted_docstring_for_symbol(symbol)
        target_file_path = (
            "/".join(f"{symbol.__module__}.{symbol.__name__}".split(".")) + ".md"
        )

        with open((docs_path / "api" / target_file_path), "w") as f:
            f.write(content)

In [ ]:
with TemporaryDirectory() as d:
    module_name = "fastkafka"

    docs_path = Path(d) / "docusaurus" / "docs"
    docs_path.mkdir(parents=True)

    api_path = docs_path / "api"
    api_path.mkdir(parents=True)

    members_with_submodules = _get_submodules(module_name)
    symbols = _load_submodules(module_name, members_with_submodules)
    for symbol in symbols:
        target_file_path = (
            "/".join(f"{symbol.__module__}.{symbol.__name__}".split(".")) + ".md"
        )
        (api_path / "/".join(f"{symbol.__module__}".split("."))).mkdir(
            parents=True, exist_ok=True
        )

        with open((api_path / target_file_path), "w") as f:
            f.write(f"Initial content in '{target_file_path}'")

        with open((api_path / target_file_path), "r") as f:
            contents = f.read()
            print(contents)
            assert f"Initial content in '{target_file_path}'" == contents, contents

    _generate_markdown_docs(module_name, docs_path)

    print("*" * 100)
    for symbol in symbols:
        target_file_path = (
            "/".join(f"{symbol.__module__}.{symbol.__name__}".split(".")) + ".md"
        )
        (api_path / "/".join(f"{symbol.__module__}".split("."))).mkdir(
            parents=True, exist_ok=True
        )

        with open((api_path / target_file_path), "r") as f:
            contents = f.read()
            print(contents)
            assert f"Initial content in '{target_file_path}'" != contents, contents

Initial content in 'fastkafka/FastKafka.md'
Initial content in 'fastkafka/KafkaEvent.md'
Initial content in 'fastkafka/testing/LocalKafkaBroker.md'
Initial content in 'fastkafka/testing/LocalRedpandaBroker.md'
Initial content in 'fastkafka/testing/Tester.md'
****************************************************************************************************
`fastkafka.FastKafka`

`def __init__(self, title: Optional[str] = None, description: Optional[str] = None, version: Optional[str] = None, contact: Optional[Dict[str, str]] = None, kafka_brokers: Dict[str, Any], root_path: Optional[pathlib.Path, str] = None, bootstrap_servers: Optional[str, List[str]] = None, lifespan: Optional[Callable[[ForwardRef('FastKafka')], AbstractAsyncContextManager[NoneType]]] = None, loop=None, client_id=None, metadata_max_age_ms=300000, request_timeout_ms=40000, api_version='auto', acks=<object object>, key_serializer=None, value_serializer=None, compression_type=None, max_batch_size=16384, partitioner=<ka

In [ ]:
# | export

_app = typer.Typer()


@_app.command()
def build_markdown_docs(
    module_name: str = typer.Option(
        None,
        help="The name of the module for which the markdown documentation should be generated. If None, then the module name will be read from settings.ini file.",
    ),
    docs_path: str = typer.Option(
        "./docusaurus/docs",
        help="The docs root path to save the generated markdown files",
    ),
) -> None:
    if module_name is None:
        module_name = get_config().lib_name

    _fix_invalid_syntax_in_markdown(Path(docs_path))
    _generate_markdown_docs(module_name, Path(docs_path))

In [ ]:
runner = CliRunner()

result = runner.invoke(_app, ["--help"])
print(result.stdout)

                                                                                                                   
 Usage: build-markdown-docs [OPTIONS]                                                                              
                                                                                                                   

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --module-name               TEXT  The name of the module for which the markdown documentation should be         │
│                                   generated. If None, then the module name will be read from settings.ini file. │
│                                   [default: None]                                                               │
│ --docs-path                 TEXT  The docs root path to save the generated markdown files                       │
│                                   [default: ./docusaurus/docs]                                                  │
│ --install-completion              Install completion for the current shell.                                     │
│ --show-completion                 Show completion for the current shell, to copy it or customize the            │
│                                   installation.                                                                 │
│ --help                            Show this message and exit.                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯